In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [20, 10]

AttributeError: module 'numpy' has no attribute '__version__'

## Train_users dataset

In [6]:
raw_user = pd.read_csv('./data/train_users_2.csv')

NameError: name 'pd' is not defined

In [ ]:
raw_user.head(10)

In [ ]:
raw_user.describe().T

In [ ]:
# ages vs date of birth
raw_user.age.isna().sum()
raw_user.age.describe()

## Session dataset

In [ ]:
raw_session = pd.read_csv('./data/sessions.csv')
raw_session.head(10)

In [ ]:
raw_session.action_detail.value_counts()

In [ ]:
raw_session.action.value_counts()

## Countries dataset

In [ ]:
raw_countries = pd.read_csv('./data/countries.csv')

In [ ]:
raw_countries.head(10)

In [ ]:
raw_countries.shape

In [ ]:
raw_countries.describe().T

In [ ]:
raw_countries.sort_values('distance_km', inplace=True)

In [ ]:
sns.barplot(x='country_destination', y='distance_km', data=raw_countries)

## Age gender bkts dataset

In [ ]:
raw_age_gender_bkts = pd.read_csv('./data/age_gender_bkts.csv')

In [ ]:
raw_age_gender_bkts.head(10)

In [ ]:
raw_age_gender_bkts.shape

In [ ]:
raw_age_gender_bkts.columns

In [ ]:
raw_age_gender_bkts.describe().T

In [ ]:
raw_age_gender_bkts['age_bucket'] = raw_age_gender_bkts['age_bucket'].str.strip()
raw_age_gender_bkts['age_bucket'].unique()
# age_range = raw_age_gender_bkts['age_bucket'].unique()

raw_age_gender_bkts['age_bucket_range'] = raw_age_gender_bkts['age_bucket'].str.split('-').apply(lambda x: int(str.strip(x[0], '+')))

In [ ]:
plt.figure(figsize=(10, 5))
sns.lmplot(x='age_bucket_range', y='population_in_thousands', data=raw_age_gender_bkts, hue='country_destination', aspect=2, size=8, fit_reg=False)

In [ ]:
sns.barplot(x='gender', y='population_in_thousands', data=raw_age_gender_bkts, hue='country_destination')

In [ ]:
raw_test_dataset = pd.read_csv('./data/test_users.csv')
raw_test_dataset.head(10)

In [ ]:
# raw_session.loc[raw_session['user_id'] == raw_test_dataset['id'][2], :]

# Preprocess train data

In [ ]:
raw_user.columns

In [ ]:
raw_test_dataset.columns

In [ ]:
raw_user_datetime = raw_user.loc[:, ['date_account_created', 'timestamp_first_active' , 'country_destination']]

In [ ]:
from datetime import datetime
# datetime format yyyy-mm-dd
func_str_datetime = lambda x: datetime.strptime(x, '%Y-%m-%d')
func_str_datetime_2 = lambda x: datetime.strptime(str(x), '%Y%m%d%H%M%S')
raw_user_datetime['date_account_created'] = raw_user_datetime['date_account_created'].apply(func_str_datetime)
raw_user_datetime['timestamp_first_active'] = raw_user_datetime['timestamp_first_active'].apply(func_str_datetime_2)

In [ ]:
raw_user_datetime.head(10)

In [ ]:
raw_user_datetime['delta_time'] = raw_user_datetime.loc[:, 'date_account_created'].dt.date - raw_user_datetime.loc[:, 'timestamp_first_active'].dt.date

In [ ]:
raw_user_datetime.describe().T

In [ ]:
raw_user_datetime.sort_values('delta_time', ascending=False).head(10)

In [ ]:
raw_user_datetime['delta_time'].dt.days.head()

In [ ]:
raw_user_datetime.head(10)

In [ ]:
raw_user_datetime.delta_time.value_counts()
raw_user_datetime.delta_time.nunique()

In [ ]:
crosstab = pd.crosstab(raw_user_datetime['delta_time'].dt.days, raw_user_datetime['country_destination'])[1:]

In [ ]:
crosstab.head(10)
# sns.tsplot(crosstab, value='NDF')

In [ ]:
crosstab.sum()

In [ ]:
crosstab.plot(kind='bar', stacked=True, colormap='tab20')

In [ ]:
crosstab_stacked = crosstab.stack().reset_index().rename(columns={0:'value'})
crosstab_stacked.head()

In [ ]:
sns.lmplot(x='delta_time', y='value', data=crosstab_stacked, hue='country_destination' ,fit_reg=False, aspect=2, size= 10, palette="muted")

In [ ]:
crosstab.reset_index().head()

In [ ]:
g = sns.FacetGrid(crosstab_stacked, row='country_destination', aspect=2)
g.map(plt.scatter, 'delta_time', 'value')

### signup_flow vs country_destination

In [ ]:
raw_user.country_destination.value_counts()

In [ ]:
raw_user.signup_flow.value_counts()

In [ ]:
sns.countplot(x='signup_flow', data=raw_user, hue='country_destination', palette='tab20')

### language vs country_destination

In [ ]:
raw_user.language.value_counts()

In [ ]:
portion_user = raw_user.sample(1000)
# lang_crosstab = pd.crosstab(raw_user.language, raw_user.country_destination)
lang_crosstab = pd.crosstab(portion_user.language, portion_user.country_destination)
lang_crosstab.head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(lang_crosstab.drop(axis=0, index='en'), ax=ax)

In [ ]:
sns.heatmap(lang_crosstab)

### 'affiliate_channel', 'affiliate_provider','first_affiliate_tracked',

In [ ]:
raw_user.affiliate_channel.value_counts()
raw_user.affiliate_channel.isna().any()
raw_user.affiliate_channel.isnull().any()
raw_user.isnull().any()

In [ ]:
sns.countplot(y='affiliate_channel', data=raw_user, hue='country_destination')

### 'signup_app', 'first_device_type', 'first_browser',

In [ ]:
raw_user.columns

In [ ]:
raw_test_dataset.signup_method.unique()

In [ ]:
# year - year
mean_age = raw_user[raw_user.age < 120].age.mean()
age_func = lambda x: 2014 - x if x > 150 else mean_age if x > 100 and x <= 150 else x

In [ ]:
raw_user.age.map(age_func).describe()

In [ ]:
sns.countplot(x=raw_user[raw_user.age > 120].age)

In [ ]:
sns.countplot(x=raw_test_dataset.age.apply(age_func))

In [ ]:
raw_test_dataset.age.apply(age_func).describe().T

## Merge session with user

In [ ]:
raw_session.head(10)

In [ ]:
raw_session.groupby(['user_id']).head(10)

In [ ]:
raw_session.columns

In [ ]:
dummies_columns = [
    'action', 
    'action_type', 
    'action_detail', 
    'device_type'
]

dummies_session = pd.get_dummies(raw_session, columns=dummies_columns)

In [ ]:
dummies_session.shape
print(dummies_session.columns.values)

In [ ]:
# dummies_session.groupby('user_id').agg(sum)

In [ ]:
dummies_session.head()

In [ ]:
dummies_session.to_csv('./data/dummies_session.csv')

In [ ]:
dummies_session.head()